In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import sys
print(sys.version)
!pip3 install ideep4py

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]


In [0]:
!git clone --branch master https://github.com/HealthplusAI/python3-gdcm.git && cd python3-gdcm && sudo dpkg -i build_1-1_amd64.deb && sudo apt-get install -f

fatal: destination path 'python3-gdcm' already exists and is not an empty directory.


In [0]:
!sudo cp /usr/local/lib/gdcm.py /usr/local/lib/python3.6/dist-packages/.
!sudo cp /usr/local/lib/gdcmswig.py /usr/local/lib/python3.6/dist-packages/.
!sudo cp /usr/local/lib/_gdcmswig.so /usr/local/lib/python3.6/dist-packages/.
!sudo cp /usr/local/lib/libgdcm* /usr/local/lib/python3.6/dist-packages/.
!ldconfig

/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [0]:
import gdcm

In [0]:
# Verify that we can access Google Drive from colab
# !ls "/content/drive/My Drive/"

In [0]:
path_base = "/content/drive/My Drive/CovidCTNet/"

In [0]:
import os
import sys
import numpy as np
import pandas as pd

In [0]:
sys.path.append('/content/drive/My Drive/CovidCTNet/preprocessing')

In [0]:
from utilities import check_paths_validity, build_patient_list, read_annotation_data, resample_ct_pixels, plot_ct_image
from utilities import truncate_hu, normalize, compute_lung_mask, apply_lung_mask, crop_ct_lungs, viz_ct_scan
from utilities import export_normal_patches, export_centered_patches, export_random_centered_patches,export_normal_slices

In [0]:
def covid_det_preprocessing(lst_patients, patient_type, train_or_test ,save_viz=False):
    print("*Starting Pre-processing")
    if patient_type=='C':
        normal_patch_shape = dct_config['covid_normal_patch_shape']
        center_patch_shape = dct_config['covid_center_patch_shape']
        annote_csv = dct_config['csv_annotation_covid']
    elif patient_type=='H':
        normal_patch_shape = dct_config['hlthy_normal_patch_shape']
        center_patch_shape = dct_config['hlthy_center_patch_shape']
        annote_csv = dct_config['csv_annotation_hlthy']
    elif patient_type=='P':
        normal_patch_shape = dct_config['pneum_normal_patch_shape']
        center_patch_shape = dct_config['pneum_center_patch_shape']
        annote_csv = dct_config['csv_annotation_pneum']

    else:
        print("Could not detect patch shape. Using default 3x128x128")
        normal_patch_shape = [3,128,128]
        center_patch_shape = [3,128,128]
        print("Could not detect annotation csv. Step 08 will be skipped")
        annote_csv = None
    if not annote_csv is None:
        # df_annotation = read_annotation_data(annote_csv)
        # lst_annot_patients = df_annotation.ID.unique().tolist()
        print('passed')
    
    num_patient = 0

    for patient in lst_patients[:]:
        patient_prefix = patient_type + '_'
        patient_id = patient_prefix + patient.strip()
        num_patient += 1
        patch_npy_prefix = patient_prefix + str(num_patient).zfill(4)

        print("\n****************************************************************")
        print("{} / {} : <{}>".format(num_patient, str(len(lst_patients)), patient_id))
        print("****************************************************************")
        if (train_or_test == 'train'):
            in_npy_ct_pixels_hu = dct_config['path_ct_pixels_hu_train'] + patient_id + '_ct-pixels.npy'
            in_npy_ct_orig_space = dct_config['path_ct_pixels_hu_train'] + patient_id + '_ct-spacing.npy'
            in_npy_ct_orig_shape = dct_config['path_ct_pixels_hu_train'] + patient_id + '_ct-orig-shape.npy'
            out_path = dct_config['path_normal_slices_train']
        else:
            in_npy_ct_pixels_hu = dct_config['path_ct_pixels_hu_test'] + patient_id + '_ct-pixels.npy'
            in_npy_ct_orig_space = dct_config['path_ct_pixels_hu_test'] + patient_id + '_ct-spacing.npy'
            in_npy_ct_orig_shape = dct_config['path_ct_pixels_hu_test'] + patient_id + '_ct-orig-shape.npy'
            out_path = dct_config['path_normal_slices_test']
        try:
            patient_ct_pixels_hu = np.load(in_npy_ct_pixels_hu)
            print("Successfully loaded:", in_npy_ct_pixels_hu)
            patient_ct_orig_space = np.load(in_npy_ct_orig_space)
            print("Successfully loaded:", in_npy_ct_orig_space)
            patient_ct_orig_shape = np.load(in_npy_ct_orig_shape)
            print("Successfully loaded:", in_npy_ct_orig_shape)
        except Exception as e:
            print(e)
            continue


        '''
        Step-01: Resample ct-pixel data
        '''
        patient_ct_resampled_hu = resample_ct_pixels(patient_ct_pixels_hu, patient_ct_orig_space)
        print("ct-resampled_hu HU range: [", np.min(patient_ct_resampled_hu), ";" , np.max(patient_ct_resampled_hu), "]")

        '''
        Step-02: Truncate HU values outside range [-1000;400]
        '''
        patient_ct_truncate_hu = truncate_hu(patient_ct_resampled_hu)
        print("ct-truncate_hu HU range: [", np.min(patient_ct_truncate_hu), ";" , np.max(patient_ct_truncate_hu), "]")

        # '''
        # Step-03: Compute binary mask for lungs
        # '''
        # if dct_config['apply_lungs_segmentation'] or dct_config['apply_cropping']:
        #     patient_ct_lung_binary_mask = compute_lung_mask(patient_ct_truncate_hu, threshold=-350)

        '''
        Step-04: Normalize
        '''
        patient_ct_norm_hu = normalize(patient_ct_truncate_hu)
        print("ct-norm-hu HU range: [", np.min(patient_ct_norm_hu), ";" , np.max(patient_ct_norm_hu), "]")    

        '''
        Step-05: Apply mask
        '''
        if dct_config['apply_lungs_segmentation']:
            patient_ct_lung_seg = apply_lung_mask(patient_ct_norm_hu, patient_ct_lung_binary_mask)
            print("ct-lung-seg HU range: [", np.min(patient_ct_lung_seg), ";" , np.max(patient_ct_lung_seg), "]")
        else:
            print("Segmentation of lungs disabled. Set dct_config['apply_lungs_segmentation'] to True to enable")

        '''
        Step-06: Crop Lung Segment
        '''
        if dct_config['apply_cropping']:
            if dct_config['apply_lungs_segmentation']: 
                patient_ct_lung_seg_cropped = crop_ct_lungs(patient_ct_lung_seg, patient_ct_lung_binary_mask, margin=32)
            else:
                patient_ct_lung_seg_cropped = crop_ct_lungs(patient_ct_norm_hu, patient_ct_lung_binary_mask, margin=32)
        else:
            print("Cropping of lungs disabled. Set dct_config['apply_cropping'] to True to enable")
        
        '''
        Step-07: Export patches(without annotation)
        '''
        if dct_config['apply_cropping']:
            # export_normal_patches(patient_ct_lung_seg_cropped,
            #                     normal_patch_shape,
            #                     dct_config['stride'],
            #                     dct_config['path_normal_patches'],
            #                     patch_npy_prefix, patient_id[2:])
            export_normal_slices(patient_ct_lung_seg_cropped,
                                normal_patch_shape,
                                dct_config['stride'],
                                out_path,
                                patch_npy_prefix, patient_id[2:])            
        else:
            # export_normal_patches(patient_ct_norm_hu, 
            #                     normal_patch_shape,
            #                     dct_config['stride'],
            #                     dct_config['path_normal_patches'],
            #                     patch_npy_prefix, patient_id[2:])          
            export_normal_slices(patient_ct_norm_hu, 
                                normal_patch_shape,
                                dct_config['stride'],
                                out_path,
                                patch_npy_prefix, patient_id[2:]) 
        '''
        # Step-08: Export centered patches(with annotation)
        '''
        # if not annote_csv is None:
        #     pat_id = patient_id[2:]
        #     if pat_id in lst_annot_patients:
        #         df_pat_annot = df_annotation[df_annotation["ID"] == pat_id]
        #         print("Number of annotations:", len(df_pat_annot))
        #         if patient_type=='C' or patient_type=='P':
        #             if dct_config['apply_lungs_segmentation']:
        #                 export_centered_patches(patient_ct_lung_seg, 
        #                                         patient_ct_orig_space, patient_ct_orig_shape,
        #                                         df_pat_annot, center_patch_shape, 
        #                                         dct_config['path_centered_patches'], 
        #                                         patch_npy_prefix, pat_id)
        #             else:
        #                 export_centered_patches(patient_ct_norm_hu, 
        #                                         patient_ct_orig_space, patient_ct_orig_shape,
        #                                         df_pat_annot, center_patch_shape, 
        #                                         dct_config['path_centered_patches'], 
        #                                         patch_npy_prefix, pat_id)                        
        #         else:
        #             if dct_config['apply_lungs_segmentation']:
        #                 export_random_centered_patches(patient_ct_lung_seg, 
        #                                         patient_ct_orig_space, patient_ct_orig_shape,
        #                                         df_pat_annot, center_patch_shape, 
        #                                         dct_config['path_centered_patches'], 
        #                                         patch_npy_prefix, pat_id)
        #             else:
        #                 export_random_centered_patches(patient_ct_norm_hu, 
        #                                         patient_ct_orig_space, patient_ct_orig_shape,
        #                                         df_pat_annot, center_patch_shape, 
        #                                         dct_config['path_centered_patches'], 
        #                                         patch_npy_prefix, pat_id)                        


        # if save_viz:
        #     viz_ct_scan(patient_ct_norm_hu, dct_config['path_debug'] + patient_id + '_normalized.pdf')
        #     viz_ct_scan(patient_ct_lung_binary_mask, dct_config['path_debug'] + patient_id + '_mask.pdf')
        #     viz_ct_scan(patient_ct_lung_seg, dct_config['path_debug'] + patient_id + '_lung_seg.pdf')
        #     viz_ct_scan(patient_ct_lung_seg_cropped, dct_config['path_debug'] + patient_id + '_lung_seg_crop.pdf')
        
    print("\n*Finished Pre-processing")

In [0]:
dct_config = {'path_ct_covid': path_base + 'Data/DCM/Covid/',
              'path_ct_hlthy': path_base + 'Data/DCM/Control/',
              'path_ct_pneum': path_base + 'Data/DCM/CAP/',
              'path_ct_test': path_base  + 'Data/DCM/TEST/',
              'csv_annotation_covid': path_base + 'Data/DCM_train_lbl/Covid19-annotations.csv',
              'csv_annotation_hlthy': path_base + 'Data/DCM_train_lbl/Healthy-annotations.csv',
              'csv_annotation_pneum': path_base + 'Data/DCM_train_lbl/Pneumonia-annotations.csv',
              'path_ct_pixels_hu_train': path_base + 'preprocessed/ct-pixels_train/',
              'path_ct_pixels_hu_test': path_base + 'preprocessed/ct-pixels_test/',

            #   'path_centered_patches': path_base + 'preprocessed/02-ct-centered-patches/',
            #   'path_normal_patches': path_base + 'preprocessed/03-ct-normal-patches/',
              'path_normal_slices_train': path_base + 'preprocessed/ct-normal-slices-train/',
              'path_normal_slices_test': path_base + 'preprocessed/ct-normal-slices-test/',
            #   'path_debug': path_base + 'preprocessed/00-debug/',
              'stride': [17,19,21], # Define the strides used to create patches
              'covid_normal_patch_shape': [3,128,128], # Define patch sizes for normal patch generation
              'hlthy_normal_patch_shape': [3,128,128],
              'pneum_normal_patch_shape': [3,128,128],
              'covid_center_patch_shape': [3,138,138], # Define patch sizes for centered annotatted patches
              'hlthy_center_patch_shape': [3,128,128],
              'pneum_center_patch_shape': [3,138,138],
              'apply_lungs_segmentation': False,
              'apply_cropping': False}

In [0]:
path_list = [path_base, 
             dct_config['path_ct_covid'], dct_config['path_ct_hlthy'], dct_config['path_ct_pneum'], dct_config['path_ct_test'],
             dct_config['path_ct_pixels_hu_train'], dct_config['path_ct_pixels_hu_test'], dct_config['path_normal_slices_train'],dct_config['path_normal_slices_test']]

# Verify all paths
check_paths_validity(path_list)

/content/drive/My Drive/CovidCTNet/  --> OK
/content/drive/My Drive/CovidCTNet/Data/DCM/Covid/  --> OK
/content/drive/My Drive/CovidCTNet/Data/DCM/Control/  --> OK
/content/drive/My Drive/CovidCTNet/Data/DCM/CAP/  --> OK
/content/drive/My Drive/CovidCTNet/Data/DCM/TEST/  --> OK
/content/drive/My Drive/CovidCTNet/preprocessed/ct-pixels_train/  --> OK
/content/drive/My Drive/CovidCTNet/preprocessed/ct-pixels_test/  --> OK
/content/drive/My Drive/CovidCTNet/preprocessed/ct-normal-slices-train/  --> OK
/content/drive/My Drive/CovidCTNet/preprocessed/ct-normal-slices-test/  --> OK


In [0]:
lst_covid_patients = build_patient_list(dct_config['path_ct_covid'],subfolder='/SR_3')
print("Total number of Covid-19  Patients: {}".format(str(len(lst_covid_patients))))
covid_det_preprocessing(lst_covid_patients,"train", "CPCR")

build_patient_list|Info: patient </content/drive/My Drive/CovidCTNet/Data/DCM/Covid/1641392+> found with 48 dcm files
build_patient_list|Info: patient </content/drive/My Drive/CovidCTNet/Data/DCM/Covid/1641266+> found with 60 dcm files
Patients detected:2
Total number of Covid-19  Patients: 2
*Starting Pre-processing
Could not detect patch shape. Using default 3x128x128
Could not detect annotation csv. Step 08 will be skipped

****************************************************************
1 / 2 : <CPCR_1641266+>
****************************************************************
Successfully loaded: /content/drive/My Drive/CovidCTNet/preprocessed/ct-pixels_train/CPCR_1641266+_ct-pixels.npy
Successfully loaded: /content/drive/My Drive/CovidCTNet/preprocessed/ct-pixels_train/CPCR_1641266+_ct-spacing.npy
Successfully loaded: /content/drive/My Drive/CovidCTNet/preprocessed/ct-pixels_train/CPCR_1641266+_ct-orig-shape.npy
resample_ct_pixels|Info ==> Original shape  : (60, 512, 512) New shape 

In [0]:
lst_hlthy_patients = build_patient_list(dct_config['path_ct_hlthy'])
print("Total number of Healthy   Patients: {}".format(str(len(lst_hlthy_patients))))
covid_det_preprocessing(lst_hlthy_patients,"train", "H")

build_patient_list|Info: patient </content/drive/My Drive/CovidCTNet/Data/DCM/Control/PATIENT 2 (4)> found with 66 dcm files
build_patient_list|Info: patient </content/drive/My Drive/CovidCTNet/Data/DCM/Control/PATIENT 2 (5)> found with 62 dcm files
Patients detected:2
Total number of Healthy   Patients: 2
*Starting Pre-processing
passed

****************************************************************
1 / 2 : <H_PATIENT 2 (4)>
****************************************************************
Successfully loaded: /content/drive/My Drive/CovidCTNet/preprocessed/ct-pixels_train/H_PATIENT 2 (4)_ct-pixels.npy
Successfully loaded: /content/drive/My Drive/CovidCTNet/preprocessed/ct-pixels_train/H_PATIENT 2 (4)_ct-spacing.npy
Successfully loaded: /content/drive/My Drive/CovidCTNet/preprocessed/ct-pixels_train/H_PATIENT 2 (4)_ct-orig-shape.npy
resample_ct_pixels|Info ==> Original shape  : (66, 512, 512) New shape  : (330, 380, 380)
resample_ct_pixels|Info ==> Original spacing: [5.   0.74 0.74] 

In [0]:
lst_pneum_patients = build_patient_list(dct_config['path_ct_pneum'])
print("Total number of Pneumonia Patients: {}".format(str(len(lst_pneum_patients))))
covid_det_preprocessing(lst_pneum_patients,"train", "P")

build_patient_list|Info: patient </content/drive/My Drive/CovidCTNet/Data/DCM/CAP/Patient_23> found with 31 dcm files
build_patient_list|Info: patient </content/drive/My Drive/CovidCTNet/Data/DCM/CAP/Patient_29> found with 40 dcm files
Patients detected:2
Total number of Pneumonia Patients: 2
*Starting Pre-processing
passed

****************************************************************
1 / 2 : <P_Patient_23>
****************************************************************
Successfully loaded: /content/drive/My Drive/CovidCTNet/preprocessed/ct-pixels_train/P_Patient_23_ct-pixels.npy
Successfully loaded: /content/drive/My Drive/CovidCTNet/preprocessed/ct-pixels_train/P_Patient_23_ct-spacing.npy
Successfully loaded: /content/drive/My Drive/CovidCTNet/preprocessed/ct-pixels_train/P_Patient_23_ct-orig-shape.npy
resample_ct_pixels|Info ==> Original shape  : (31, 512, 512) New shape  : (310, 384, 384)
resample_ct_pixels|Info ==> Original spacing: [10.    0.75  0.75] New spacing: [1. 1. 1.]

In [0]:
lst_test_patients = build_patient_list(dct_config['path_ct_test'])
print("Total number of TEST  Patients: {}".format(str(len(lst_test_patients))))
covid_det_preprocessing(lst_test_patients,"test", "T")

build_patient_list|Info: patient </content/drive/My Drive/CovidCTNet/Data/DCM/TEST/TEST (5)> found with 60 dcm files
build_patient_list|Info: patient </content/drive/My Drive/CovidCTNet/Data/DCM/TEST/TEST (1)> found with 341 dcm files
Patients detected:2
Total number of TEST  Patients: 2
*Starting Pre-processing
Could not detect patch shape. Using default 3x128x128
Could not detect annotation csv. Step 08 will be skipped

****************************************************************
1 / 2 : <T_TEST (1)>
****************************************************************
Successfully loaded: /content/drive/My Drive/CovidCTNet/preprocessed/ct-pixels_test/T_TEST (1)_ct-pixels.npy
Successfully loaded: /content/drive/My Drive/CovidCTNet/preprocessed/ct-pixels_test/T_TEST (1)_ct-spacing.npy
Successfully loaded: /content/drive/My Drive/CovidCTNet/preprocessed/ct-pixels_test/T_TEST (1)_ct-orig-shape.npy
resample_ct_pixels|Info ==> Original shape  : (341, 512, 512) New shape  : (341, 383, 383)
